In [2]:
import os
import pyspedas
from pyspedas import mms_load_fpi,tinterpol
from pytplot import tplot, get_data, store_data, tlimit, options,tplot_names
from pyspedas.mms import fpi,fgm
from pyspedas.mms.mms_orbit_plot import mms_orbit_plot
import plotly.express as px
import matplotlib.pyplot as plt


from pyspedas import time_string
import pandas as pd
import numpy as np


import pyspedas
import pytplot

save_fig = 'no'
# save_csv = 'no'

save_images = 'no'
save_csv = 'no'


den_csv = 'MMS2_ni.csv'
mag_arr_str = 'MMS2_B.csv'
v_arr_str = 'MMS2_Vi.csv'

temp_perp = 'MMS2_Tiperp.csv'

temp_para = 'MMS2_Tipara.csv'

pressure_anisotropy = 'pressure_anisotropy.csv'

alpha = pd.read_csv(pressure_anisotropy,header = 0,names = ['times','alpha','date'],
                         parse_dates=['date'])
mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])
v_arr = pd.read_csv(v_arr_str,header = 0,names = ['times','Vix','Viy','Viz','date'],
                         parse_dates=['date'])

MMS2_ni = pd.read_csv(den_csv,header = 0,names = ['times','Ni','date'],
                         parse_dates=['date'])



Tipara_csv = pd.read_csv(temp_para,header = 0,names = ['times','Tipara','date'],
                         parse_dates=['date'])

Tiperp_csv = pd.read_csv(temp_perp,header = 0,names = ['times','Tiperp','date'],
                         parse_dates=['date'])

In [9]:
 def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return int(idx)

In [35]:
# MSH_times =np.array([['2020-06-17/23:11:10.000','2020-06-17/23:14:40.000'],
# ['2020-06-17/23:27:40','2020-06-17/23:30:00'],
#                      ['2020-06-17/22:00:50','2020-06-17/22:04:50'],
#                      ['2020-06-17/22:16:40','2020-06-17/22:18:30'],
#                       ['2020-06-17/22:24:10','2020-06-17/22:27:50'],
#                      ['2020-06-17/23:31:10','2020-06-17/23:34:10'],
#                      ['2020-06-17/23:05:00','2020-06-17/23:08:50'],
#                     ['2020-06-17/23:27:50','2020-06-17/23:30:30'],
#                     ['2020-06-18/00:05:00','2020-06-18/00:08:30']])


MSH_times =np.array([['2020-06-17/23:50:40.000','2020-06-17/23:53:30.000'],
['2020-06-17/23:58:40','2020-06-18/00:00:30'],
                     ['2020-06-17/22:31:30','2020-06-17/22:34:00'],
                     ['2020-06-18/00:19:50','2020-06-18/00:21:40'],
                      ['2020-06-18/00:41:20','2020-06-18/00:42:50']])

MSP_times =np.array([['2020-06-17/22:04:40','2020-06-17/22:11:10'],
                     ['2020-06-17/22:20:30','2020-06-17/22:23:50'],
                     ['2020-06-17/22:28:00','2020-06-17/22:30:40'],
                     ['2020-06-17/22:34:30','2020-06-17/22:37:10'],
                     ['2020-06-17/22:39:20','2020-06-17/22:43:30'],
                     ['2020-06-17/22:56:30','2020-06-17/23:00:30'],
                     ['2020-06-17/23:31:50','2020-06-17/23:36:50'],
                     ['2020-06-17/23:46:10','2020-06-17/23:50:20'],
                     ['2020-06-17/23:54:00','2020-06-17/23:57:50']])



In [36]:
# find average density during MSH times


MSH_density=[]
MSP_density=[]
MSH_temp=[]
MSP_temp=[]
for i in range(len(MSH_times)):
    t1 = pd.to_datetime(MSH_times[i,0])
    t2 = pd.to_datetime(MSH_times[i,1])
    a = (MMS2_ni['Ni'][MMS2_ni.date.searchsorted(t1):MMS2_ni.date.searchsorted(t2)]).to_list()
    MSH_density = np.hstack((MSH_density,a))
    b = (Tipara_csv['Tipara'][Tipara_csv.date.searchsorted(t1):Tipara_csv.date.searchsorted(t2)]).to_list()
    MSH_temp =np.hstack((MSH_temp,b))   

for i in range(len(MSP_times)):
    t1 = pd.to_datetime(MSP_times[i,0])
    t2 = pd.to_datetime(MSP_times[i,1])
    a = (MMS2_ni['Ni'][MMS2_ni.date.searchsorted(t1):MMS2_ni.date.searchsorted(t2)]).to_list()
    MSP_density = np.hstack((MSP_density,a))
    b = (Tipara_csv['Tipara'][Tipara_csv.date.searchsorted(t1):Tipara_csv.date.searchsorted(t2)]).to_list()
    MSP_temp =np.hstack((MSP_temp,b))   

print(MSH_density)
# for t in MSH_density:
#     print(t,'\n')
#d = MSH_density.reshape(1,len())
# print(len(MSH_density))

# MSH_density
# d = np.array(MSH_density)
# g = d.reshape((1, len(d)))

# g
ave_MSH_density = np.mean(np.array(MSH_density))
ave_MSP_density = np.mean(np.array(MSP_density))
std_MSH_density = np.std(np.array(MSH_density))
std_MSP_density = np.std(np.array(MSP_density))

print('average MSH density:',ave_MSH_density,'with std:',std_MSH_density)
print('average MSP density:',ave_MSP_density,'with std:',std_MSP_density)

ave_MSH_temp = np.mean(MSH_temp)
ave_MSP_temp = np.mean(MSP_temp)
std_MSH_temp = np.std(MSH_temp)
std_MSP_temp = np.std(MSP_temp)

print('average MSH temp:',ave_MSH_temp,'with std:',std_MSH_temp)
print('average MSP temp:',ave_MSP_temp,'with std:',std_MSP_temp)


[1.7740052 1.7467403 1.9046342 ... 2.1317565 2.1765792 2.1410317]
average MSH density: 4.99240215993811 with std: 1.882945663600451
average MSP density: 1.121887237353103 with std: 0.2131335920727534
average MSH temp: 114.53928914996429 with std: 47.35173020792942
average MSP temp: 402.2399125294482 with std: 114.37249644975454


In [31]:
1400+934+1600+733+1466+1200+1533+1067+1400

11333

In [11]:
np.mean(MSH_density)

array([0.10866518, 0.10375413, 0.10249854, ..., 0.18626898, 0.18009907,
       0.16882229])

# Alternative method loading the data from pyspead

In [3]:
# other method
ion_vars = pyspedas.mms.mms_load_fpi(trange=['2020-06-17/20:30:00','2020-06-18/01:30:00'],probe='2',data_rate='fast',datatype='dis-moms',  center_measurement=True,time_clip=True)


# Vi_mms2 = 'mms2_dis_bulkv_gse_brst'
# times_vi_mms2, data_vi_mms2 = get_data(Vi_mms2)


# closest_MSH_times = []
# for i in range(len(MSH_times)):
#     closest_MSH_times.append(find_nearest(times_vi_mms2,peak_times[i]))


28-Oct-24 18:59:51: Loading pydata/mms2/fpi/fast/l2/dis-moms/2020/06/mms2_fpi_fast_l2_dis-moms_20200617200000_v3.4.0.cdf
28-Oct-24 18:59:51: Loading pydata/mms2/fpi/fast/l2/dis-moms/2020/06/mms2_fpi_fast_l2_dis-moms_20200617220000_v3.4.0.cdf
28-Oct-24 18:59:51: Loading pydata/mms2/fpi/fast/l2/dis-moms/2020/06/mms2_fpi_fast_l2_dis-moms_20200618000000_v3.4.0.cdf
28-Oct-24 18:59:56: Time clip was applied to: mms2_dis_errorflags_fast
28-Oct-24 18:59:56: Time clip was applied to: mms2_dis_startdelphi_count_fast
28-Oct-24 18:59:56: Time clip was applied to: mms2_dis_startdelphi_angle_fast
28-Oct-24 18:59:56: Time clip was applied to: mms2_dis_energyspectr_px_fast
28-Oct-24 18:59:56: Time clip was applied to: mms2_dis_energyspectr_mx_fast
28-Oct-24 18:59:56: Time clip was applied to: mms2_dis_energyspectr_py_fast
28-Oct-24 18:59:56: Time clip was applied to: mms2_dis_energyspectr_my_fast
28-Oct-24 18:59:56: Time clip was applied to: mms2_dis_energyspectr_pz_fast
28-Oct-24 18:59:56: Time clip 

In [7]:
MSH_times = np.array([[1592432210.0000000,1592432340.0000000],[1592432685.0000000,1592432860.0000000],
                      [1592433050.0000000,1592433240.0000000],[1592433455.0000000,1592433560.0000000],
                      [1592433830.0000000,1592433990.0000000],[1592434390.0000000,1592434545.0000000],
                      [1592434850.0000000,1592434980.0000000],[1592435110.0000000,1592435325.0000000],
                      [1592435470.0000000,1592435660.0000000],[1592435800.0000000,1592435900.0000000],
                      [1592435945.0000000,1592436160.0000000],[1592436470.0000000,1592436625.0000000],
                      [1592437025.0000000,1592437285.0000000],[1592437400.0000000,1592437575.0000000],
                      [1592437825.0000000,1592438030.0000000],
                      [1592438295.0000000,1592438475.0000000],[1592438700.0000000,1592438910.0000000],
                      [1592439085.0000000,1592439140.0000000],[1592439215.0000000,1592439370.0000000],
                      [1592439555.0000000,1592439705.0000000],[1592439845.0000000,1592440060.0000000]])

MSP_times = np.array([[1592432865.0000000,1592433050.0000000],[1592433255.0000000,1592433450.0000000],
                     [1592433565.0000000,1592433800.0000000],[1592434005.0000000,1592434175.0000000],
                     [1592434560.0000000,1592434840.0000000],[1592435345.0000000,1592435460.0000000],
                     [1592436160.0000000,1592436415.0000000],[1592437285.0000000,1592437385.0000000],
                     [1592437580.0000000,1592437820.0000000],[1592438030.0000000,1592438280.0000000],
                     [1592438505.0000000,1592438695.0000000],[1592438915.0000000,1592439080.0000000],
                     [1592439380.0000000,1592439540.0000000],[1592440060.0000000,1592440440.0000000]])

ni_mms2 = 'mms2_dis_numberdensity_fast'
Ti_para_mms2 = 'mms2_dis_temppara_fast'
Ti_perp_mms2 = 'mms2_dis_tempperp_fast'

times_ni_mms2, data_ni_mms2 = get_data(ni_mms2)

times_Ti_para_mms2, data_Tipara_mms2 = get_data(Ti_para_mms2)
times_Ti_perp_mms2, data_Tiperp_mms2 = get_data(Ti_perp_mms2)

print(data_Tipara_mms2)
print(data_Tiperp_mms2)
ave_ti = (data_Tipara_mms2+data_Tiperp_mms2)/2



[464.9085  454.4872  446.7163  ... 272.0428  292.22873 300.3668 ]
[469.46658 442.9997  432.5193  ... 261.4589  285.93552 295.81934]
[467.18756 448.74347 439.6178  ... 266.75085 289.08212 298.09308]


In [19]:
MSH_ni = []
MSH_Ti = []
for i in range(len(MSH_times)):
    st_t = find_nearest(times_ni_mms2, MSH_times[i,0])
    end_t = find_nearest(times_ni_mms2, MSH_times[i,1])
    MSH_ni_vals = data_ni_mms2[st_t:end_t]
    MSH_ni = np.hstack((MSH_ni_vals,MSH_ni))
    MSH_Ti_vals = ave_ti[st_t:end_t]
    MSH_Ti = np.hstack((MSH_Ti_vals,MSH_Ti))



MSP_ni = []
MSP_Ti = []
for i in range(len(MSP_times)):
    st_t = find_nearest(times_ni_mms2, MSP_times[i,0])
    end_t = find_nearest(times_ni_mms2, MSP_times[i,1])
    MSP_ni_vals = data_ni_mms2[st_t:end_t]
    MSP_ni = np.hstack((MSP_ni_vals,MSP_ni))
    MSP_Ti_vals = ave_ti[st_t:end_t]
    MSP_Ti = np.hstack((MSP_Ti_vals,MSP_Ti))


In [20]:
MSH_ni

array([ 0.90135306,  0.93566084,  1.17649269,  1.38224435,  2.37822938,
        3.3190577 ,  3.5312717 ,  5.37522936,  5.05216122,  5.3810463 ,
        4.94175959,  5.3276825 ,  4.96560955,  4.70641565,  4.48920393,
        4.6798625 ,  5.1017828 ,  5.20654869,  5.05969524,  4.9586587 ,
        5.08462095,  4.78591919,  4.74377537,  4.32510328,  4.54488134,
        4.59223747,  4.32685328,  3.76839495,  4.05846548,  4.22874498,
        4.36122513,  3.51201797,  3.42619514,  3.28902507,  3.19339728,
        3.16819477,  1.94853759,  1.7058636 ,  1.72813177,  1.49841213,
        1.25855339,  1.2563951 ,  1.51146328,  1.48540533,  1.55602694,
        1.59698856,  1.37103248,  1.25243568,  1.69058883,  1.2861346 ,
        1.1044513 ,  1.45282829,  2.3370986 ,  2.62327528,  1.68373907,
        1.26947474,  1.22166634,  4.07283926,  7.99981976,  7.91768074,
        7.42765427,  6.77583647,  6.9718833 ,  7.0381875 ,  6.58315468,
        4.48145056,  5.77528048,  6.970366  ,  5.77511883,  3.89

In [22]:
ave_MSH_density = np.mean(np.array(MSH_ni))
ave_MSP_density = np.mean(np.array(MSP_ni))
std_MSH_density = np.std(np.array(MSH_ni))
std_MSP_density = np.std(np.array(MSP_ni))

print('average MSH density:',ave_MSH_density,'with std:',std_MSH_density)
print('average MSP density:',ave_MSP_density,'with std:',std_MSP_density)

ave_MSH_temp = np.mean(MSH_Ti)
ave_MSP_temp = np.mean(MSP_Ti)
std_MSH_temp = np.std(MSH_Ti)
std_MSP_temp = np.std(MSP_Ti)

print('average MSH temp:',ave_MSH_temp,'with std:',std_MSH_temp)
print('average MSP temp:',ave_MSP_temp,'with std:',std_MSP_temp)


average MSH density: 4.107189718316744 with std: 1.86492738797212
average MSP density: 1.1130345533551713 with std: 0.18944988796018133
average MSH temp: 123.70139345219962 with std: 67.75849847471328
average MSP temp: 367.3786993746765 with std: 94.66737509671054


In [25]:

print('high-end MSH temperature values',ave_MSH_temp+std_MSH_temp)
print('low-end MSH temperature values',ave_MSH_temp-std_MSH_temp)

high-end MSH temperature values 191.45989192691292
low-end MSH temperature values 55.94289497748635


In [26]:
print('high-end MSP temperature values',ave_MSP_temp+std_MSP_temp)
print('low-end MSP temperature values',ave_MSP_temp-std_MSP_temp)

high-end MSP temperature values 462.04607447138704
low-end MSP temperature values 272.71132427796596
